In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchmetrics.classification import MulticlassAccuracy

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
from tqdm import tqdm 
import pickle
import functools

from concurrent import futures 

In [2]:
class NTUDataset(Dataset):
    def __init__(self, x, y):
        self.x_data = x
        self.y_data = y
        
    def __len__(self):
        return len(self.x_data)
    def __getitem__(self, i):
        return self.x_data[i], self.y_data[i]

In [ ]:
class NTUDatasetGenerator:
    def quaternion_matrix(q):
        """Return homogeneous rotation matrix from quaternion.
        """
        qw = q[0]
        qx = q[1]
        qy = q[2]
        qz = q[3]

        return np.array([
          [1-2*qy*qy - 2*qz*qz,       2*qx*qy - 2*qz*qw,        2*qx*qz + 2*qy*qw],
          [2*qx*qy + 2*qz*qw    ,     1-2*qx*qx - 2*qz*qz,      2*qy*qz - 2*qx*qw],
          [2*qx*qz - 2*qy*qw    ,     2*qy*qz + 2*qx*qw,        1-2*qx*qx - 2*qy*qy],
          ])

    def matrix_quaternion(R):  
        trace = np.trace(R)
        if trace > 0:
            s = 0.5 / np.sqrt(trace+1.0)
            qw = 0.25 / s 
            qx = (R[2,1]-R[1,2])*s
            qy = (R[0,2]-R[2,0])*s
            qz = (R[1,0]-R[0,1])*s
        else:
            if ( R[0,0] > R[1,1] and R[0,0] > R[2,2] ):
                s = 2.0 * np.sqrt( 1.0 + R[0,0] - R[1,1] - R[2,2])
                qw = (R[2,1] - R[1,2]) / s
                qx = 0.25 * s
                qy = (R[0,1] + R[1,0]) / s
                qz = (R[0,2] + R[2,0]) / s
            elif (R[1,1] > R[2,2]):
                s = 2.0 * np.sqrt( 1.0 + R[1,1] - R[0,0] - R[2,2])
                qw = (R[0,2] - R[2,0] ) / s
                qx = (R[0,1] + R[1,0] ) / s
                qy = 0.25 * s
                qz = (R[1,2] + R[2,1] ) / s
            else:
                s = 2.0 * np.sqrt( 1.0 + R[2,2] - R[0,0] - R[1,1] )
                qw = (R[1,0] - R[0,1] ) / s
                qx = (R[0,2] + R[2,0] ) / s
                qy = (R[1,2] + R[2,1] ) / s
                qz = 0.25 * s

        return np.array([qw,qx,qy,qz])
    
    # 카메라로 측정한 좌표계를 앵커(허리 중심부)를 기준으로 재구성
    def normalize_skeleton(jinfo,anchor=None,norm_dist=0,right_to_left=None,spine_to_top=None):
        _EPS = np.finfo(float).eps * 4.0
        if type(anchor)==type(None) and norm_dist == 0:
            anchor = np.array([jinfo[1][0],jinfo[1][1],jinfo[1][2]])
            base = np.array([jinfo[0][0],jinfo[0][1],jinfo[0][2]])
            norm_dist = np.linalg.norm(anchor-base) # base와 anchor의 거리를 1로 설정


        ## TRANSLATE TO SPINE ORIGIN FIRST  
        norm_joints = {}
        for jnum, unnorm_joint in iter(jinfo.items()):
            normalized_pos = np.array([unnorm_joint[0]-anchor[0],
                              unnorm_joint[1]-anchor[1],
                              unnorm_joint[2]-anchor[2]]) #/ norm_dist
            norm_joints[jnum] = normalized_pos

        # get shoulder direction unit vector
        if right_to_left is None:
            right_to_left = np.array([norm_joints[8][0],norm_joints[8][1],norm_joints[8][2]]) - np.array([norm_joints[4][0],norm_joints[4][1],norm_joints[4][2]])
            right_to_left = right_to_left / (np.linalg.norm(right_to_left)+_EPS)


        ## COMPUTE ROTATION SUCH THAT RIGHT TO LEFT IS PARALLEL TO X_AXIS
        # 어깨를 기준으로 한 새로운 좌표계 구성
        x_axis = np.array([1,0,0])
        y_axis = np.array([0,1,0])

        new_x = right_to_left # (3, )
        new_y = np.cross(right_to_left,x_axis)
        new_y = new_y/(np.linalg.norm(new_y)+_EPS)
        new_z = np.cross(new_x,new_y)
        new_z = new_z/(np.linalg.norm(new_z)+_EPS)
        Rx = np.transpose(np.array([new_x,new_y,new_z])) # 어깨 축을 기반으로 한 새로운 좌표계 Transform matrix (normal vector)
        #pdb.set_trace()


        rotated_and_norm = {}
        # 왜 dot..?
        for jnum, joint in iter(norm_joints.items()):
            turn_to_x = np.dot(joint,Rx)
            #turn_to_y = np.dot(Ry,turn_to_x)
            rotated_and_norm[jnum] = turn_to_x 

        new_right_to_left = np.array([rotated_and_norm[8][0],rotated_and_norm[8][1],rotated_and_norm[8][2]]) - np.array([rotated_and_norm[4][0],rotated_and_norm[4][1],rotated_and_norm[4][2]])

        # x축 정렬 이후 y축 기준 정렬
        if spine_to_top is None:
            spine_to_top = np.array([rotated_and_norm[0][0],rotated_and_norm[0][1],rotated_and_norm[0][2]]) - np.array([rotated_and_norm[1][0],rotated_and_norm[1][1],rotated_and_norm[1][2]])
            spine_to_top = spine_to_top / (np.linalg.norm(spine_to_top)+_EPS)


        ## COMPUTE ROTATION SUCH THAT SPINE TO TOP IS PARALLEL TO Y_AXIS
        new_y = spine_to_top
        new_x = np.cross(spine_to_top,y_axis)
        new_x = new_x/(np.linalg.norm(new_x)+_EPS)
        new_z = np.cross(new_x,new_y)
        new_z = new_z/(np.linalg.norm(new_z)+_EPS)
        Ry = np.transpose(np.array([new_x,new_y,new_z]))


        rotated_and_norm2 = {}
        for jnum, joint in iter(rotated_and_norm.items()):
        #turn_to_x = np.dot(Rx,joint)
            turn_to_y = np.dot(joint,Ry)
            # 카메라 좌표계 기준 쿼터니언 
            qR = NTUDatasetGenerator.quaternion_matrix(jinfo[jnum][3:])
            
            # 현 좌표계 기준 쿼터니언 변환
            rotated_q = np.dot(np.dot(qR,Rx),Ry)
            qq = NTUDatasetGenerator.matrix_quaternion(rotated_q)

            normed_final_vec = turn_to_y / (norm_dist+_EPS)

            rotated_and_norm2[jnum] = np.concatenate((normed_final_vec,qq))

        return rotated_and_norm2,anchor,norm_dist,right_to_left,spine_to_top

    def getfiles(self, lines, file_name):
        try:
            if not '.skeleton' in file_name:
    #             print('ret state 1')
                return 0, 0, 0

    #             print(file_name)

            action_class = int(file_name[file_name.find('A')+1:file_name.find('A')+4])
            subject_id = int(file_name[file_name.find('S')+1:file_name.find('S')+4])


            ptr = 0
            num_frames = int(lines[ptr]); ptr+=1
            feature = np.zeros((num_frames, self.feat_dim))
            for n in range(0,num_frames):
                body_count = int(lines[ptr]); ptr+=1
              #print body_count

                if body_count > 2:
                # JUST ITERATE THROUGH THE LINES, IGNORE 
                    for b in range(0,body_count):
                        body_info = lines[ptr]; ptr+=1
                        joint_count = int(lines[ptr]); ptr+=1
                        for j in range(0,joint_count):
                            joint_info = lines[ptr]; ptr+=1
                else:
                    binfo = dict()
                    norm_dist = 0
                    anchor = None
                    right_to_left = None
                    spine_to_top = None
                    for b in range(0,body_count):
                        body_info = lines[ptr]; ptr+=1
                        bsp = body_info.split()

                        body_id = bsp[0]
                        cliped_edges = bsp[1]
                        lefthand_confidence = bsp[2]
                        lefthand_state = bsp[3]
                        righthand_confidence = bsp[4]
                        righthand_state = bsp[5]
                        is_restricted = bsp[6]
                        lean_x = bsp[7]
                        lean_y = bsp[8]
                        body_tracking_state = bsp[9]

                        #binfo[b] = bsp
                        joint_count = int(lines[ptr]); ptr+=1 ## ASSUMING THIS IS ALWAYS 25

                        jinfo = dict()

                        for j in range(0,joint_count):
                            joint_info = lines[ptr]; ptr+=1
                            jsp = joint_info.split()
                            x = float(jsp[0])
                            y = float(jsp[1])
                            z = float(jsp[2])
                            depth_x = float(jsp[3])
                            depth_y = float(jsp[4])
                            rgb_x = float(jsp[5])
                            rgb_y = float(jsp[6])
                            rw = float(jsp[7])
                            rx = float(jsp[8])
                            ry = float(jsp[9])
                            rz = float(jsp[10])
                            joint_tracking_state = jsp[11]

                            jinfo[j] = (x,y,z,rw,rx,ry,rz)
                        ## END JOINT LOOP


                        norm_jinfo,anchor,norm_dist,right_to_left,spine_to_top = NTUDatasetGenerator.normalize_skeleton(jinfo,anchor=anchor,norm_dist=norm_dist,right_to_left=right_to_left,spine_to_top=spine_to_top)

                        binfo[b] = norm_jinfo

                    ## END BODY LOOP
                    sample_ind = 0
                    sample = np.zeros(self.feat_dim)

                    ## CONSTRUCT THE FEATURE FOR THIS N-th FRAME
                    for bind, body in iter(binfo.items()):
                        for jind, joint in iter(body.items()):
                            sample[sample_ind] =   joint[0] #x
                            sample_ind += 1
                            sample[sample_ind] =   joint[1] #y
                            sample_ind += 1
                            sample[sample_ind] =   joint[2] #z
                            sample_ind += 1
                    feature[n] = sample
              ## END BODY COUNT IF-ELSE
            ## END FRAME LOOP
            training_subjects = [1,2,4,5,8,9,13,14,15,16,17,18,19,25,27,28,31,34,35,38]
            if body_count <= 2:
                if subject_id in training_subjects:
                    return 1, feature, action_class-1
                else:
                    return 2, feature, action_class-1
        except Exception as e:
            print(e)
    def read(self, file_name):
        sf = open(os.path.join(self.root, file_name),'r')
        data = (file_name, sf.readlines())
        sf.close()
        return data
    
    def setLineset(self, lineset):
        self.lineset = lineset
        
    def makeLineset(self):
        self.filepath = os.listdir(self.root)
        bad_file = open('D:/datasets/NTU_RGB+D/NTU_RGBD120_missings.txt', 'r')
        bad_list = [ line.strip() + '.skeleton' for line in bad_file]
        bad_file.close()
        
        
        self.filepath = list(set(self.filepath) - set(bad_list)) 
        
        self.lineset = []
        print('start load files')
        with futures.ThreadPoolExecutor(max_workers=16) as executor:
            with tqdm(total=len(self.filepath)) as progress:
                fs = []
                for file in self.filepath:
                    ft = executor.submit(self.read, file)
                    ft.add_done_callback(lambda p: progress.update())
                    fs.append(ft)
                self.lineset = list(
                    filter(
                        lambda item: item is not None, 
                        map(
                            lambda x : x.result(), 
                            fs
                        )
                    )
                )
        
    def makeDataset(self):
        max_vid_length = -1
        X_train = []
        X_test = []
        y_train = []
        y_test = []
        n_classes = 60
        with futures.ThreadPoolExecutor(max_workers=16) as executor:
            with tqdm(total=len(self.lineset)) as progress:
                fs = []
                
                for (file_name, lines) in self.lineset:
                    ft = executor.submit(self.getfiles, lines, file_name)
                    ft.add_done_callback(lambda p: progress.update())
                    fs.append(ft)
                
                res = list(
                    filter(
                        lambda item: item is not None, 
                        map(lambda x: x.result(), fs)
                    )
                )

                for t, x, y in res:
                    if t == 0:
                        continue
                    elif t == 1:
                        X_train.append(x)
                        y_train.append(y)
                    else:
                        X_test.append(x)
                        y_test.append(y)

        self.train_dataset = NTUDataset(X_train, y_train)
        self.test_dataset = NTUDataset(X_test, y_test)
    
    def __init__(self):
        
        self._EPS = np.finfo(float).eps * 4.0
        self.feat_dim = 150 
        self.root = 'D:/datasets/NTU_RGB+D/cl60/nturgb+d_skeletons'
        
#         self.makeLineset()
    
#         print('complete loading files')
        
#         self.makeDataset()
#         print('complete making files')
        

In [ ]:
dataset_gen = NTUDatasetGenerator()

In [ ]:
dataset_gen.makeLineset()

In [ ]:
dataset_gen.setLineset(ls)

In [ ]:
ls = dataset_gen.lineset

In [ ]:
dataset_gen.makeDataset()

In [ ]:
with open('dataset_test.pkl', 'wb') as f:
    pickle.dump(dataset_gen.test_dataset, f)

In [ ]:
with open('dataset_train.pkl', 'wb') as f:
    pickle.dump(dataset_gen.train_dataset, f)

In [ ]:
with open('dataset_train.pkl', 'rb') as train_f:
    train_dataset = pickle.load(train_f)
with open('dataset_test.pkl', 'rb') as test_f:
    test_dataset = pickle.load(test_f)


In [3]:
import lmdb

In [6]:
del train_x, train_y, test_x, test_y

In [4]:
data_root = 'd:/datasets/NTU_RGB+D/subjects_split_raw/'
lmdb_file_train_x = os.path.join(data_root,'Xtrain_lmdb')
lmdb_file_train_y = os.path.join(data_root,'Ytrain_lmdb')
lmdb_file_test_x = os.path.join(data_root,'Xtest_lmdb')
lmdb_file_test_y = os.path.join(data_root,'Ytest_lmdb')

train_x_mean = 0.5909148848768809
samples_per_epoch = 39889
samples_per_validation = 16390

tqdm.clear
train_x = []
train_y = []
test_x = []
test_y = []
with lmdb.open(lmdb_file_train_x) as lmdb_env_x:
    lmdb_txn_x = lmdb_env_x.begin()
    lmdb_cursor_x = lmdb_txn_x.cursor()
    train_x = [np.frombuffer(lmdb_cursor_x.get('{:0>8d}'.format(index).encode())) for index in tqdm(np.arange(0, samples_per_epoch))]
with lmdb.open(lmdb_file_train_y) as lmdb_env_y:
    lmdb_txn_y = lmdb_env_y.begin()
    lmdb_cursor_y = lmdb_txn_y.cursor()
    train_y = [np.frombuffer(lmdb_cursor_y.get('{:0>8d}'.format(index).encode())) for index in tqdm(np.arange(0, samples_per_epoch))]

with lmdb.open(lmdb_file_test_x) as lmdb_env_x:
    lmdb_txn_x = lmdb_env_x.begin()
    lmdb_cursor_x = lmdb_txn_x.cursor()
    test_x = [np.frombuffer(lmdb_cursor_x.get('{:0>8d}'.format(index).encode())) for index in tqdm(np.arange(0, samples_per_validation))]

with lmdb.open(lmdb_file_test_y) as lmdb_env_y:
    lmdb_txn_y = lmdb_env_y.begin()
    lmdb_cursor_y = lmdb_txn_y.cursor()
    test_y = [np.frombuffer(lmdb_cursor_y.get('{:0>8d}'.format(index).encode())) for index in tqdm(np.arange(0, samples_per_validation))]


100%|████████████████████████████████████████████████████████████████████████| 16390/16390 [00:00<00:00, 199720.06it/s]


In [7]:
train_dataset = NTUDataset(train_x, train_y)
test_dataset = NTUDataset(test_x, test_y)

In [ ]:
train_x[1].shape
train_x_mean = 0.5909148848768809

In [5]:
np.random.seed(seed = 1234) 
def collate_fn(batch): # 32
    target = np.zeros((len(batch), 150, 300))
#     target = np.zeros((len(batch), 22, 300))
#     print(target.shape)
    ylist= []
    cnt = 0
    for (x, y) in batch:
        x = np.array(x).reshape((300, 150))
        nonzeros = np.where(np.array([np.sum(x[i])>0 for i in range(0,x.shape[0])])==False)[0]
        if len(nonzeros) == 0:
            last_time = 0
        else:
            last_time = nonzeros[0]
        x.setflags(write=True)
        x[:last_time] = x[:last_time] - train_x_mean # 동영상의 모든 프레임에 대해 정규화
        
        
#         print(x.shape)
        target[cnt, :, :x.shape[0]] = x.T
        ylist.append(y)
        cnt+=1 
#     print(target.shape)
    return torch.FloatTensor(target), torch.FloatTensor(ylist)



In [8]:
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=128, collate_fn=collate_fn)

In [ ]:
torch.FloatTensor(test_dataset[0][1]).max(dim=0)[1].item()

In [ ]:
next(iter(test_dataloader))[1].shape

In [32]:
class ResBlock(nn.Module):
    def __init__(
        self, 
        in_channels, 
        out_channels, 
        stride=1,
        downsample = None,
        norm_layer = None
    ):
        super(ResBlock, self).__init__()
        
        if norm_layer == None:
            norm_layer = nn.BatchNorm1d
        
        self.layer1 = nn.Sequential(
            norm_layer(in_channels),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Conv1d(
                in_channels=in_channels, 
                out_channels=out_channels, 
                kernel_size=8, 
                stride=stride, 
                padding=('same' if stride == 1 else 3)
            )
        )
#         self.relu = nn.ReLU(inplace=True)
#         self.conv = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=(3, 3), stride=1, padding=1)
#         self.bn = norm_layer(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        identity = x
        
#         print('--> id shape: ', x.shape)
        
        out = self.layer1(x)
        if self.downsample is not None:
            identity = self.downsample(x)
#             print('----> down shape: ', identity.shape)
        
#         print('--> out shape: ', out.shape)
        
        out += identity
        
        return out 

class ResTCN(nn.Module):
    def __init__(self, max_len=300, feat_dim=150, debug=False):
        super(ResTCN, self).__init__()
        self.debug = debug
        self.input_layer = nn.Conv1d(in_channels=feat_dim, out_channels=64, kernel_size=8, padding='same')
        
        downsample = lambda i, o: nn.Sequential(
                nn.Conv1d(i, o, kernel_size=1, stride=2, padding=0),
#                 nn.BatchNorm1d(o, momentum=0.99, eps=1e-3),
            )
#         self.layer = nn.Sequential(
#             ResBlock(64, 150,  stride = 1, downsample=dd(64, 150)), *[ 
#             ResBlock(150, 150,  stride = 1) for _ in range(2)
#         ])
        self.layer1 = nn.Sequential(
            ResBlock(64, 64,  stride = 1),
            ResBlock(64, 64,  stride = 1),
            ResBlock(64, 64,  stride = 1)
        )
        self.layer2 = nn.Sequential(
            ResBlock(64, 128, 2, downsample=downsample(64, 128)),
            ResBlock(128, 128, 1),
            ResBlock(128, 128, 1)
        )
        self.layer3 = nn.Sequential(
            ResBlock(128, 256, 2, downsample=downsample(128, 256)),
            ResBlock(256, 256, 1),
            ResBlock(256, 256, 1)
        )
        
        self.layer_final = nn.Sequential(
            nn.BatchNorm1d(256),
            nn.ReLU()
        )
        
        self.avgpool = nn.AvgPool1d(75, stride=1, )
        
        self.fc = nn.Sequential(
            nn.Linear(256, 60),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        if self.debug:
            print('input: ', x.shape)
        out = self.input_layer(x)
#         if self.debug: 
#             print('layer input: ',out.shape)
#         out = self.layer(out)

        if self.debug:
            print('layer1 input: ',out.shape)
        out = self.layer1(out)
        
        if self.debug:
            print('layer2 input: ', out.shape)
        out = self.layer2(out)
        
        if self.debug:
            print('layer3 input: ', out.shape)
        out = self.layer3(out)
        
        if self.debug:
            print('layer final input: ', out.shape)
        out = self.layer_final(out)
    
        out = torch.mean(out, dim=-1, keepdim=False)
    
#         if self.debug:
#             print('layer avg pool input: ', out.shape)        
#         out = self.avgpool(out)
        
#         if self.debug:
#             print('layer flatten input: ', out.shape)
#         out = torch.flatten(out, 1)
        
        if self.debug:
            print('layer fc input: ', out.shape)
        
        return self.fc(out)

In [33]:
def initialize_weights(m):
    if isinstance(m, nn.Conv1d):
        nn.init.kaiming_normal_(m.weight.data,nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm1d):
        nn.init.constant_(m.weight.data, 1)
        nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight.data)
        nn.init.constant_(m.bias.data, 0)

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
ex_input = torch.randn((10, 32, 10)).to(device)
# m = nn.AvgPool2d(kernel_size = (256, 1), stride = (1, 1))
m = nn.AvgPool1d(10).to(device)
ex_output = m(ex_input)
print(ex_output.shape)
print(ex_input)
print(ex_output)

In [30]:
import torchsummary
torchsummary.summary(ResTCN(feat_dim=150,debug=True).to(device), input_size=(150, 300), batch_size=128)

input:  torch.Size([2, 150, 300])
layer1 input:  torch.Size([2, 64, 300])
layer2 input:  torch.Size([2, 64, 300])


RuntimeError: The size of tensor a (149) must match the size of tensor b (150) at non-singleton dimension 2

In [27]:
model.layer_final[0].bias.shape

torch.Size([256])

In [ ]:
tmp = next(iter(train_dataloader))[1]

In [ ]:
print(tmp.shape)
print(tmp.max(dim=1)[1].shape)

In [15]:
def model_train(model:ResTCN, data_loader, loss_fn, optimizer, device):
    model.train()
    
    running_size = 0
    running_loss = 0
    corr = 0
    
    prograss_bar = tqdm(data_loader)
    
    total_corr = 0
    
    # mini-batch 학습을 시작합니다.
    for batch_idx, (img, lbl) in enumerate(prograss_bar, start=1):
        img, lbl = img.to(device), lbl.to(device)
        optimizer.zero_grad()
        
        output = model(img)
        
        loss = loss_fn(output, lbl) # Cross Entropy with Sum over 
#         l1_lambda = 1e-4
#         l1_norm = sum(torch.linalg.norm(w.reshape((-1, 1)), 1) for w in [model.input_layer.weight,
#             model.layer1[0].layer1[3].weight,
#             model.layer1[1].layer1[3].weight,
#             model.layer1[2].layer1[3].weight,
#             model.layer2[0].layer1[3].weight,
#             model.layer2[0].downsample[0].weight,
#             model.layer2[1].layer1[3].weight,
#             model.layer2[2].layer1[3].weight,
#             model.layer3[0].layer1[3].weight,
#             model.layer3[0].downsample[0].weight,
#             model.layer3[1].layer1[3].weight,
#             model.layer3[2].layer1[3].weight,])
        
#         loss = loss + (l1_lambda * l1_norm).to(device)
        
        loss.backward()
        
        optimizer.step()
        pred = output.argmax(dim=1)
        label = lbl.argmax(dim=1)
        
#         corr += pred.eq(label).sum().item()
#         corr = metric(pred, label)
        
        running_loss += loss.item()
        running_size += img.size(0)
        total_corr += (pred == label).sum().item()
        
#         prograss_bar.set_description(f'[Training] loss: {running_loss / running_size:.4f}, accuracy: {corr / running_size:.4f}')
        prograss_bar.set_description(f'[Training] loss: {running_loss / running_size:.4f}, accuracy: {total_corr/running_size:.4f}')
        
    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
#     acc = corr / len(data_loader.dataset)
    acc = total_corr / len(data_loader.dataset)
    
    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader.dataset), acc

In [16]:
[model.input_layer.weight,
model.layer1[0].layer1[3].weight,
model.layer1[1].layer1[3].weight,
model.layer1[2].layer1[3].weight,
model.layer2[0].layer1[3].weight,
model.layer2[0].downsample[0].weight,
model.layer2[1].layer1[3].weight,
model.layer2[2].layer1[3].weight,
model.layer3[0].layer1[3].weight,
model.layer3[0].downsample[0].weight,
model.layer3[1].layer1[3].weight,
model.layer3[2].layer1[3].weight,]

NameError: name 'model' is not defined

In [ ]:
(img, lbl) = next(iter(test_dataloader))

In [ ]:
metric = MulticlassAccuracy(num_classes=60, average='macro').to(device)

img, lbl = img.to(device), lbl.to(device)
output = model(img)
corr1 = metric(output, lbl.argmax(dim=1))
corr2 = metric(output.argmax(dim=1), lbl.argmax(dim=1))
print(output.argmax(dim=1))
print(lbl.argmax(dim=1))
print(corr1.shape, corr1)
print(corr2.shape, corr2)
print((output.argmax(dim=1) == lbl.argmax(dim=1)).sum().item()/128)

In [ ]:
met = MulticlassAccuracy(num_classes=5, average='none')
print(met(torch.FloatTensor([1, 1, 2, 3, 0]), torch.LongTensor([3, 1, 2, 0, 4])))

In [17]:
def model_evaluate(model, data_loader, loss_fn, device):
    # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다. 
    # dropout과 같은 layer의 역할 변경을 위하여 evaluation 진행시 꼭 필요한 절차 입니다.
    model.eval()
    prograss_bar = tqdm(data_loader)
    
    # Gradient가 업데이트 되는 것을 방지 하기 위하여 반드시 필요합니다.
    with torch.no_grad():
        # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
        corr = 0
        running_loss = 0
        running_size = 0
        
        total_corr = 0
        # 배치별 evaluation을 진행합니다.
        for batch_idx, (img, lbl) in enumerate(prograss_bar, start=1):
            # device에 데이터를 올립니다.
            img, lbl = img.to(device), lbl.to(device)
            
            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = model(img)
            
            pred = output.argmax(dim=1)
            label = lbl.argmax(dim=1)

            running_loss += loss_fn(output, lbl).item()
            running_size += img.size(0)
            total_corr += (pred == label).sum().item()

    #         prograss_bar.set_description(f'[Training] loss: {running_loss / running_size:.4f}, accuracy: {corr / running_size:.4f}')
            prograss_bar.set_description(f'[Test] loss: {running_loss / running_size:.4f}, accuracy: {total_corr/running_size:.4f}')

        # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    #     acc = corr / len(data_loader.dataset)
        
        acc = total_corr / len(data_loader.dataset)
#         sample = output.argmax(dim=1) == lbl.argmax(dim=1)
#         print(sample.shape, sample.sum().item())
        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader.dataset), acc

In [18]:
CNT = 0

In [35]:
LR = 1e-4
NUM_EPOCHS = 200

model = ResTCN(feat_dim=150).to(device)
model.apply(initialize_weights)

# 옵티마이저를 정의합니다. 옵티마이저에는 model.parameters()를 지정해야 합니다.
# optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=0.0, nesterov=True)
optimizer = optim.Adam(model.parameters(), lr=LR)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, cooldown=3, min_lr=1e-4, verbose=True)
# scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=1e-4, )

loss_fn = nn.CrossEntropyLoss(reduction='sum')

In [36]:
import wandb

MODEL_NAME = f'ResTCN_{CNT}_{LR}_HE_Origin_Final'

wandb.init(
    # set the wandb project where this run will be logged
    project="RVP",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": LR,
    "architecture": MODEL_NAME,
    "epochs": NUM_EPOCHS,
    }
)
CNT += 1

train_acc,▁▂▂▃▃▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████
train_loss,█▇▇▆▆▆▅▅▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_acc,▁▂▂▃▃▃▄▄▄▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▆▇▇▇▇▇▇██████
val_loss,█▇▇▇▆▆▆▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▃▂▂▂▂▂▂▁▁▁▁▁▁
train_acc,0.30297
train_loss,3.83158
val_acc,0.29079
val_loss,3.83851


In [37]:
wandb.run.name = f'OUR_ResTCN_New_try_{CNT}'

min_loss = np.inf
for epoch in range(NUM_EPOCHS):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    train_loss, train_acc = model_train(model, train_dataloader, loss_fn, optimizer, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate(model, test_dataloader, loss_fn, device)
    
    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(model.state_dict(), f'{MODEL_NAME}.pth')
    scheduler.step(val_loss)
    
    wandb.log({"val_acc": val_acc, "val_loss": val_loss, 'train_acc':train_acc, 'train_loss' : train_loss})
    
    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

[Test] loss: 4.0552, accuracy: 0.0701: 100%|█████████████████████████████████████████| 129/129 [00:29<00:00,  4.43it/s]


[INFO] val_loss has been improved from inf to 4.05515. Saving Model!
epoch 01, loss: 4.06633, acc: 0.05941, val_loss: 4.05515, val_accuracy: 0.07010


[Test] loss: 4.0379, accuracy: 0.0996: 100%|█████████████████████████████████████████| 129/129 [00:27<00:00,  4.71it/s]


[INFO] val_loss has been improved from 4.05515 to 4.03785. Saving Model!
epoch 02, loss: 4.04071, acc: 0.08348, val_loss: 4.03785, val_accuracy: 0.09957


[Test] loss: 4.0222, accuracy: 0.1075: 100%|█████████████████████████████████████████| 129/129 [00:26<00:00,  4.90it/s]


...

epoch 198, loss: 3.59954, acc: 0.52844, val_loss: 3.65468, val_accuracy: 0.47059


[Test] loss: 3.6406, accuracy: 0.4843: 100%|█████████████████████████████████████████| 129/129 [00:22<00:00,  5.63it/s]


epoch 199, loss: 3.59832, acc: 0.53000, val_loss: 3.64058, val_accuracy: 0.48432


[Test] loss: 3.6510, accuracy: 0.4743: 100%|█████████████████████████████████████████| 129/129 [00:24<00:00,  5.31it/s]

epoch 200, loss: 3.59759, acc: 0.53082, val_loss: 3.65099, val_accuracy: 0.47425


In [ ]:
[p.shape for p in [*model.parameters()][:-2] if len(p.shape) == 3 and p.shape[-1] == 8]

In [ ]:
a = torch.FloatTensor([[5, 1., 2.,10,  3. ,4.]])
_, pred = a.max(dim=1)
print(pred)

In [ ]:
a = torch.randn((10, 64, 300))

In [ ]:
nn.BatchNorm1d(64).weight.shape

In [ ]:
(8, ) + (150, 64)